In [0]:
import logging
logger = spark._jvm.org.apache.log4j
logging.getLogger("py4j").setLevel(logging.ERROR)

In [0]:
import xarray as xr
import xhydro as xh
import xdatasets as xd
import numpy as np
import xhydro.frequency_analysis as xhfa
from scipy.stats.mstats import plotting_positions

In [0]:
ds = xd.Query(
    **{
        "datasets":{
            "deh":{
                "id" :["020*"],
                "regulated":["Natural"],
                "variables":["streamflow"],
            }
        }, "time":{"start": "1970-01-01",
                   "minimum_duration":(15*365, 'd')},

  }
).data.squeeze().load()

# This dataset lacks some of the aforementioned attributes, so we need to add them.
ds["id"].attrs["cf_role"] = "timeseries_id"
ds["streamflow"].attrs = {"long_name": "Streamflow", "units": "m3 s-1", "standard_name": "water_volume_transport_in_river_channel", "cell_methods": "time: mean"}

ds

Out[8]:

<xarray.Dataset>
Dimensions: (id: 5, time: 20454)
Coordinates: (12/15)
 drainage_area (id) float32 1.09e+03 647.0 59.8 626.0 1.2e+03
 end_date (id) datetime64[ns] 2006-10-13 2024-01-28 ... 1996-08-13
 * id (id) object '020302' '020404' '020502' '020602' '020802'
 latitude (id) float32 48.77 48.81 48.98 48.98 49.2
 longitude (id) float32 -64.52 -64.92 -64.43 -64.7 -65.29
 name (id) object 'Saint' 'York' ... 'Dartmouth' 'Madeleine'
 ... ...
 spatial_agg <U9 'watershed'
 start_date (id) datetime64[ns] 1989-08-12 1980-10-01 ... 1970-01-01
 * time (time) datetime64[ns] 1970-01-01 1970-01-02 ... 2025-12-31
 time_agg <U4 'mean'
 timestep <U1 'D'
 variable <U10 'streamflow'
Data variables:
 streamflow (id, time) float32 nan nan nan nan nan ... nan nan nan nan xarray.Dataset Dimensions: id : 5 time : 20454 Coordinates: (15) drainage_area (id) float32 1.09e+03 647.0 59.8 626.0 1.2e+03 long_name : drainage_area units : km2 array([1090. , 647. , 59.8, 626. , 1200. ], dtype=float32) end_date (id) datetime64[ns] 2006-10-13 ... 1996-08-13 array(['2006-10-13T00:00:00.000000000', '2024-01-28T00:00:00.000000000',
 '1997-06-08T00:00:00.000000000', '2024-01-28T00:00:00.000000000',
 '1996-08-13T00:00:00.000000000'], dtype='datetime64[ns]') id (id) object '020302' '020404' ... '020802' cf_role : timeseries_id array(['020302', '020404', '020502', '020602', '020802'], dtype=object) latitude (id) float32 48.77 48.81 48.98 48.98 49.2 long_name : latitude standard_name : latitude units : decimal_degrees array([48.769165, 48.806946, 48.982224, 48.97778 , 49.202778],
 dtype=float32) longitude (id) float32 -64.52 -64.92 -64.43 -64.7 -65.29 long_name : longitude standard_name : longitude units : decimal_degrees array([-64.51583 , -64.916664, -64.42694 , -64.69972 , -65.29472 ],
 dtype=float32) name (id) object 'Saint' 'York' ... 'Madeleine' array(['Saint', 'York', 'Au Renard', 'Dartmouth', 'Madeleine'],
 dtype=object) province (id) object 'QC' 'QC' 'QC' 'QC' 'QC' array(['QC', 'QC', 'QC', 'QC', 'QC'], dtype=object) regulated (id) object 'Natural' 'Natural' ... 'Natural' array(['Natural', 'Natural', 'Natural', 'Natural', 'Natural'],
 dtype=object) source () <U102 'Ministère de l’Environnement, d... array('Ministère de l’Environnement, de la Lutte contre les changements climatiques, de la Faune et des Parcs',
 dtype='<U102') spatial_agg () <U9 'watershed' array('watershed', dtype='<U9') start_date (id) datetime64[ns] 1989-08-12 ... 1970-01-01 array(['1989-08-12T00:00:00.000000000', '1980-10-01T00:00:00.000000000',
 '1977-07-10T00:00:00.000000000', '1970-10-01T00:00:00.000000000',
 '1970-01-01T00:00:00.000000000'], dtype='datetime64[ns]') time (time) datetime64[ns] 1970-01-01 ... 2025-12-31 array(['1970-01-01T00:00:00.000000000', '1970-01-02T00:00:00.000000000',
 '1970-01-03T00:00:00.000000000', ..., '2025-12-29T00:00:00.000000000',
 '2025-12-30T00:00:00.000000000', '2025-12-31T00:00:00.000000000'],
 dtype='datetime64[ns]') time_agg () <U4 'mean' array('mean', dtype='<U4') timestep () <U1 'D' array('D', dtype='<U1') variable () <U10 'streamflow' array('streamflow', dtype='<U10') Data variables: (1) streamflow (id, time) float32 nan nan nan nan ... nan nan nan nan long_name : Streamflow units : m3 s-1 standard_name : water_volume_transport_in_river_channel cell_methods : time: mean array([[ nan, nan, nan, ..., nan, nan, nan],
 [ nan, nan, nan, ..., nan, nan, nan],
 [ nan, nan, nan, ..., nan, nan, nan],
 [ nan, nan, nan, ..., nan, nan, nan],
 [14.7, 14.3, 14. , ..., nan, nan, nan]], dtype=float32) Indexes: (2) id PandasIndex PandasIndex(Index(['020302', '020404', '020502', '020602', '020802'], dtype='object', name='id')) time PandasIndex PandasIndex(DatetimeIndex(['1970-01-01', '1970-01-02', '1970-01-03', '1970-01-04',
 '1970-01-05', '1970-01-06', '1970-01-07', '1970-01-08',
 '1970-01-09', '1970-01-10',
 ...
 '2025-12-22', '2025-12-23', '2025-12-24', '2025-12-25',
 '2025-12-26', '2025-12-27', '2025-12-28', '2025-12-29',
 '2025-12-30', '2025-12-31'],
 dtype='datet

In [0]:
# Some examples
timeargs = {
    "spring": {"date_bounds": ["02-11", "06-19"]},
    "summer": {"doy_bounds": [152, 243]},
    "fall": {"month": [9, 10, 11]},
    "winter": {"season": ['DJF'], "freq": "AS-DEC"},  # To correctly wrap around the year, we need to specify the resampling frequency.
    "august": {"month": [8]},
    "annual": {}
    }

In [0]:
# Here, we hide years with more than 15% of missing data.
ds_4fa = xh.indicators.get_yearly_op(ds, op="max", timeargs=timeargs, missing="pct", missing_options={"tolerance": 0.15})

ds_4fa

Out[10]:

<xarray.Dataset>
Dimensions: (id: 5, time: 56)
Coordinates: (12/15)
 drainage_area (id) float32 1.09e+03 647.0 59.8 626.0 1.2e+03
 end_date (id) datetime64[ns] 2006-10-13 ... 1996-08-13
 * id (id) object '020302' '020404' ... '020602' '020802'
 latitude (id) float32 48.77 48.81 48.98 48.98 49.2
 longitude (id) float32 -64.52 -64.92 -64.43 -64.7 -65.29
 name (id) object 'Saint' 'York' ... 'Madeleine'
 ... ...
 spatial_agg <U9 'watershed'
 start_date (id) datetime64[ns] 1989-08-12 ... 1970-01-01
 time_agg <U4 'mean'
 timestep <U1 'D'
 variable <U10 'streamflow'
 * time (time) datetime64[ns] 1970-01-01 ... 2025-01-01
Data variables:
 streamflow_max_spring (id, time) float32 nan nan nan nan ... nan nan nan
 streamflow_max_summer (id, time) float32 nan nan nan nan ... nan nan nan
 streamflow_max_fall (id, time) float32 nan nan nan nan ... nan nan nan
 streamflow_max_august (id, time) float32 nan nan nan nan ... nan nan nan
 streamflow_max_annual (id, time) float32 nan nan nan nan ... nan nan nan
 streamflow_max_winter (id, time) float32 nan nan nan nan ... nan nan nan
Attributes:
 cat:frequency: yr
 cat:processing_level: indicators
 cat:id: xarray.Dataset Dimensions: id : 5 time : 56 Coordinates: (15) drainage_area (id) float32 1.09e+03 647.0 59.8 626.0 1.2e+03 long_name : drainage_area units : km2 array([1090. , 647. , 59.8, 626. , 1200. ], dtype=float32) end_date (id) datetime64[ns] 2006-10-13 ... 1996-08-13 array(['2006-10-13T00:00:00.000000000', '2024-01-28T00:00:00.000000000',
 '1997-06-08T00:00:00.000000000', '2024-01-28T00:00:00.000000000',
 '1996-08-13T00:00:00.000000000'], dtype='datetime64[ns]') id (id) object '020302' '020404' ... '020802' cf_role : timeseries_id array(['020302', '020404', '020502', '020602', '020802'], dtype=object) latitude (id) float32 48.77 48.81 48.98 48.98 49.2 long_name : latitude standard_name : latitude units : decimal_degrees array([48.769165, 48.806946, 48.982224, 48.97778 , 49.202778],
 dtype=float32) longitude (id) float32 -64.52 -64.92 -64.43 -64.7 -65.29 long_name : longitude standard_name : longitude units : decimal_degrees array([-64.51583 , -64.916664, -64.42694 , -64.69972 , -65.29472 ],
 dtype=float32) name (id) object 'Saint' 'York' ... 'Madeleine' array(['Saint', 'York', 'Au Renard', 'Dartmouth', 'Madeleine'],
 dtype=object) province (id) object 'QC' 'QC' 'QC' 'QC' 'QC' array(['QC', 'QC', 'QC', 'QC', 'QC'], dtype=object) regulated (id) object 'Natural' 'Natural' ... 'Natural' array(['Natural', 'Natural', 'Natural', 'Natural', 'Natural'],
 dtype=object) source () <U102 'Ministère de l’Environnement, d... array('Ministère de l’Environnement, de la Lutte contre les changements climatiques, de la Faune et des Parcs',
 dtype='<U102') spatial_agg () <U9 'watershed' array('watershed', dtype='<U9') start_date (id) datetime64[ns] 1989-08-12 ... 1970-01-01 array(['1989-08-12T00:00:00.000000000', '1980-10-01T00:00:00.000000000',
 '1977-07-10T00:00:00.000000000', '1970-10-01T00:00:00.000000000',
 '1970-01-01T00:00:00.000000000'], dtype='datetime64[ns]') time_agg () <U4 'mean' array('mean', dtype='<U4') timestep () <U1 'D' array('D', dtype='<U1') variable () <U10 'streamflow' array('streamflow', dtype='<U10') time (time) datetime64[ns] 1970-01-01 ... 2025-01-01 array(['1970-01-01T00:00:00.000000000', '1971-01-01T00:00:00.000000000',
 '1972-01-01T00:00:00.000000000', '1973-01-01T00:00:00.000000000',
 '1974-01-01T00:00:00.000000000', '1975-01-01T00:00:00.000000000',
 '1976-01-01T00:00:00.000000000', '1977-01-01T00:00:00.000000000',
 '1978-01-01T00:00:00.000000000', '1979-01-01T00:00:00.000000000',
 '1980-01-01T00:00:00.000000000', '1981-01-01T00:00:00.000000000',
 '1982-01-01T00:00:00.000000000', '1983-01-01T00:00:00.000000000',
 '1984-01-01T00:00:00.000000000', '1985-01-01T00:00:00.000000000',
 '1986-01-01T00:00:00.000000000', '1987-01-01T00:00:00.000000000',
 '1988-01-01T00:00:00.000000000', '1989-01-01T00:00:00.000000000',
 '1990-01-01T00:00:00.000000000', '1991-01-01T00:00:00.000000000',
 '199

In [0]:
from scipy.stats.mstats import plotting_positions
def get_plotting_positions(data, alpha=0.4, beta=0.4, return_period=True):
  def vec_plotting_positions(vec_data,  alpha=0.4, beta=0.4):
    out = []
    for data in vec_data:
      data[~np.isnan(data)] = plotting_positions(data[~np.isnan(data)])
      out.append(data)
    return out
  if return_period:
    return -1 /((xr.apply_ufunc(vec_plotting_positions, ds_4fa, alpha, beta, input_core_dims=[['time'], [], []], output_core_dims=[['time']]))-1)
  else:
    return xr.apply_ufunc(vec_plotting_positions, ds_4fa, alpha, beta, input_core_dims=[['time'], [], []], output_core_dims=[['time']])


In [0]:
pp = get_plotting_positions(ds_4fa, alpha=0.1, beta=0.1)
pp

Out[79]:

<xarray.Dataset>
Dimensions: (id: 5, time: 56)
Coordinates: (12/15)
 drainage_area (id) float32 1.09e+03 647.0 59.8 626.0 1.2e+03
 end_date (id) datetime64[ns] 2006-10-13 ... 1996-08-13
 * id (id) object '020302' '020404' ... '020602' '020802'
 latitude (id) float32 48.77 48.81 48.98 48.98 49.2
 longitude (id) float32 -64.52 -64.92 -64.43 -64.7 -65.29
 name (id) object 'Saint' 'York' ... 'Madeleine'
 ... ...
 spatial_agg <U9 'watershed'
 start_date (id) datetime64[ns] 1989-08-12 ... 1970-01-01
 time_agg <U4 'mean'
 timestep <U1 'D'
 variable <U10 'streamflow'
 * time (time) datetime64[ns] 1970-01-01 ... 2025-01-01
Data variables:
 streamflow_max_spring (id, time) float32 nan nan nan nan ... nan nan nan
 streamflow_max_summer (id, time) float32 nan nan nan nan ... nan nan nan
 streamflow_max_fall (id, time) float32 nan nan nan nan ... nan nan nan
 streamflow_max_august (id, time) float32 nan nan nan nan ... nan nan nan
 streamflow_max_annual (id, time) float32 nan nan nan nan ... nan nan nan
 streamflow_max_winter (id, time) float32 nan nan nan nan ... nan nan nan xarray.Dataset Dimensions: id : 5 time : 56 Coordinates: (15) drainage_area (id) float32 1.09e+03 647.0 59.8 626.0 1.2e+03 long_name : drainage_area units : km2 array([1090. , 647. , 59.8, 626. , 1200. ], dtype=float32) end_date (id) datetime64[ns] 2006-10-13 ... 1996-08-13 array(['2006-10-13T00:00:00.000000000', '2024-01-28T00:00:00.000000000',
 '1997-06-08T00:00:00.000000000', '2024-01-28T00:00:00.000000000',
 '1996-08-13T00:00:00.000000000'], dtype='datetime64[ns]') id (id) object '020302' '020404' ... '020802' cf_role : timeseries_id array(['020302', '020404', '020502', '020602', '020802'], dtype=object) latitude (id) float32 48.77 48.81 48.98 48.98 49.2 long_name : latitude standard_name : latitude units : decimal_degrees array([48.769165, 48.806946, 48.982224, 48.97778 , 49.202778],
 dtype=float32) longitude (id) float32 -64.52 -64.92 -64.43 -64.7 -65.29 long_name : longitude standard_name : longitude units : decimal_degrees array([-64.51583 , -64.916664, -64.42694 , -64.69972 , -65.29472 ],
 dtype=float32) name (id) object 'Saint' 'York' ... 'Madeleine' array(['Saint', 'York', 'Au Renard', 'Dartmouth', 'Madeleine'],
 dtype=object) province (id) object 'QC' 'QC' 'QC' 'QC' 'QC' array(['QC', 'QC', 'QC', 'QC', 'QC'], dtype=object) regulated (id) object 'Natural' 'Natural' ... 'Natural' array(['Natural', 'Natural', 'Natural', 'Natural', 'Natural'],
 dtype=object) source () <U102 'Ministère de l’Environnement, d... array('Ministère de l’Environnement, de la Lutte contre les changements climatiques, de la Faune et des Parcs',
 dtype='<U102') spatial_agg () <U9 'watershed' array('watershed', dtype='<U9') start_date (id) datetime64[ns] 1989-08-12 ... 1970-01-01 array(['1989-08-12T00:00:00.000000000', '1980-10-01T00:00:00.000000000',
 '1977-07-10T00:00:00.000000000', '1970-10-01T00:00:00.000000000',
 '1970-01-01T00:00:00.000000000'], dtype='datetime64[ns]') time_agg () <U4 'mean' array('mean', dtype='<U4') timestep () <U1 'D' array('D', dtype='<U1') variable () <U10 'streamflow' array('streamflow', dtype='<U10') time (time) datetime64[ns] 1970-01-01 ... 2025-01-01 array(['1970-01-01T00:00:00.000000000', '1971-01-01T00:00:00.000000000',
 '1972-01-01T00:00:00.000000000', '1973-01-01T00:00:00.000000000',
 '1974-01-01T00:00:00.000000000', '1975-01-01T00:00:00.000000000',
 '1976-01-01T00:00:00.000000000', '1977-01-01T00:00:00.000000000',
 '1978-01-01T00:00:00.000000000', '1979-01-01T00:00:00.000000000',
 '1980-01-01T00:00:00.000000000', '1981-01-01T00:00:00.000000000',
 '1982-01-01T00:00:00.000000000', '1983-01-01T00:00:00.000000000',
 '1984-01-01T00:00:00.000000000', '1985-01-01T00:00:00.000000000',
 '1986-01-01T00:00:00.000000000', '1987-01-01T00:00:00.000000000',
 '1988-01-01T00:00:00.000000000', '1989-01-01T00:00:00.000000000',
 '1990-01-01T00:00:00.000000000', '1991-01-01T00:00:00.000000000',
 '1992-01-01T00:00:00.000000000', '1993-01-01T00:00:00.000000000',
 '1994-01-01

In [0]:
params = xhfa.local.fit(ds_4fa)
params

Out[60]:

<xarray.Dataset>
Dimensions: (id: 5, dparams: 5, scipy_dist: 7)
Coordinates: (12/16)
 * id (id) object '020302' '020404' ... '020602' '020802'
 * dparams (dparams) <U5 'a' 'c' 'skew' 'loc' 'scale'
 * scipy_dist (scipy_dist) <U11 'expon' 'gamma' ... 'weibull_min'
 drainage_area (id) float32 dask.array<chunksize=(5,), meta=np.ndarray>
 end_date (id) datetime64[ns] dask.array<chunksize=(5,), meta=np.ndarray>
 latitude (id) float32 dask.array<chunksize=(5,), meta=np.ndarray>
 ... ...
 source <U102 'Ministère de l’Environnement, de la Lutte c...
 spatial_agg <U9 'watershed'
 start_date (id) datetime64[ns] dask.array<chunksize=(5,), meta=np.ndarray>
 time_agg <U4 'mean'
 timestep <U1 'D'
 variable <U10 'streamflow'
Data variables:
 streamflow_max_spring (scipy_dist, id, dparams) float64 dask.array<chunksize=(1, 5, 5), meta=np.ndarray>
 streamflow_max_summer (scipy_dist, id, dparams) float64 dask.array<chunksize=(1, 5, 5), meta=np.ndarray>
 streamflow_max_fall (scipy_dist, id, dparams) float64 dask.array<chunksize=(1, 5, 5), meta=np.ndarray>
 streamflow_max_august (scipy_dist, id, dparams) float64 dask.array<chunksize=(1, 5, 5), meta=np.ndarray>
 streamflow_max_annual (scipy_dist, id, dparams) float64 dask.array<chunksize=(1, 5, 5), meta=np.ndarray>
 streamflow_max_winter (scipy_dist, id, dparams) float64 dask.array<chunksize=(1, 5, 5), meta=np.ndarray>
Attributes:
 cat:frequency: yr
 cat:processing_level: indicators
 cat:id: xarray.Dataset Dimensions: id : 5 dparams : 5 scipy_dist : 7 Coordinates: (16) id (id) object '020302' '020404' ... '020802' cf_role : timeseries_id array(['020302', '020404', '020502', '020602', '020802'], dtype=object) dparams (dparams) <U5 'a' 'c' 'skew' 'loc' 'scale' array(['a', 'c', 'skew', 'loc', 'scale'], dtype='<U5') scipy_dist (scipy_dist) <U11 'expon' 'gamma' ... 'weibull_min' array(['expon', 'gamma', 'genextreme', 'genpareto', 'gumbel_r', 'pearson3',
 'weibull_min'], dtype='<U11') drainage_area (id) float32 dask.array<chunksize=(5,), meta=np.ndarray> long_name : drainage_area units : km2 
 
 
 
 
 
 
 Array 
 Chunk 
 
 
 
 
 
 Bytes 
 20 B 
 20 B 
 
 
 
 Shape 
 (5,) 
 (5,) 
 
 
 Dask graph 
 1 chunks in 16 graph layers 
 
 
 Data type 
 float32 numpy.ndarray 
 
 
 
 
 
 

 <!-- Horizontal lines -->
 
 

 <!-- Vertical lines -->
 
 

 <!-- Colored Rectangle -->
 

 <!-- Text -->
 5 
 1 
 
 
 
 end_date (id) datetime64[ns] dask.array<chunksize=(5,), meta=np.ndarray> 
 
 
 
 
 
 
 Array 
 Chunk 
 
 
 
 
 
 Bytes 
 40 B 
 40 B 
 
 
 
 Shape 
 (5,) 
 (5,) 
 
 
 Dask graph 
 1 chunks in 16 graph layers 
 
 
 Data type 
 datetime64[ns] numpy.ndarray 
 
 
 
 
 
 

 <!-- Horizontal lines -->
 
 

 <!-- Vertical lines -->
 
 

 <!-- Colored Rectangle -->
 

 <!-- Text -->
 5 
 1 
 
 
 
 latitude (id) float32 dask.array<chunksize=(5,), meta=np.ndarray> long_name : latitude standard_name : latitude units : decimal_degrees 
 
 
 
 
 
 
 Array 
 Chunk 
 
 
 
 
 
 Bytes 
 20 B 
 20 B 
 
 
 
 Shape 
 (5,) 
 (5,) 
 
 
 Dask graph 
 1 chunks in 16 graph layers 
 
 
 Data type 
 float32 numpy.ndarray 
 
 
 
 
 
 

 <!-- Horizontal lines -->
 
 

 <!-- Vertical lines -->
 
 

 <!-- Colored Rectangle -->
 

 <!-- Text -->
 5 
 1 
 
 
 
 longitude (id) float32 dask.array<chunksize=(5,), meta=np.ndarray> long_name : longitude standard_name : longitude units : decimal_degrees 
 
 
 
 
 
 
 Array 
 Chunk 
 
 
 
 
 
 Bytes 
 20 B 
 20 B 
 
 
 
 Shape 
 (5,) 
 (5,) 
 
 
 Dask graph 
 1 chunks in 16 graph layers 
 
 
 Data type 
 float32 numpy.ndarray 
 
 
 
 
 
 

 <!-- Horizontal lines -->
 
 

 <!-- Vertical lines -->
 
 

 <!-- Colored Rectangle -->
 

 <!-- Text -->
 5 
 1 
 
 
 
 name (id) object dask.array<chunksize=(5,), meta=np.ndarray> 
 
 
 
 
 
 
 Array 
 Chunk 
 
 
 
 
 
 Bytes 
 40 B 
 40 B 
 
 
 
 Shape 
 (5,) 
 (5,) 
 
 
 Dask graph 
 1 chunks in 16 graph layers 
 
 
 Data type 
 object numpy.ndarray 
 
 
 
 
 
 

 <!-- Horizontal lines -->
 
 

 <!-- Vertical lines -->
 
 

 <!-- Colored Rectangle -->
 

 <!-- 

In [0]:
x = np.linspace(1.01,10000,100)
x

Out[62]: array([1.01000000e+00, 1.02009899e+02, 2.03009798e+02, 3.04009697e+02,
 4.05009596e+02, 5.06009495e+02, 6.07009394e+02, 7.08009293e+02,
 8.09009192e+02, 9.10009091e+02, 1.01100899e+03, 1.11200889e+03,
 1.21300879e+03, 1.31400869e+03, 1.41500859e+03, 1.51600848e+03,
 1.61700838e+03, 1.71800828e+03, 1.81900818e+03, 1.92000808e+03,
 2.02100798e+03, 2.12200788e+03, 2.22300778e+03, 2.32400768e+03,
 2.42500758e+03, 2.52600747e+03, 2.62700737e+03, 2.72800727e+03,
 2.82900717e+03, 2.93000707e+03, 3.03100697e+03, 3.13200687e+03,
 3.23300677e+03, 3.33400667e+03, 3.43500657e+03, 3.53600646e+03,
 3.63700636e+03, 3.73800626e+03, 3.83900616e+03, 3.94000606e+03,
 4.04100596e+03, 4.14200586e+03, 4.24300576e+03, 4.34400566e+03,
 4.44500556e+03, 4.54600545e+03, 4.64700535e+03, 4.74800525e+03,
 4.84900515e+03, 4.95000505e+03, 5.05100495e+03, 5.15200485e+03,
 5.25300475e+03, 5.35400465e+03, 5.45500455e+03, 5.55600444e+03,
 5.65700434e+03, 5.75800424e+03, 5.85900414e+03, 5.96000404e+03,
 6.06100394e+03, 6.16200384e+03, 6.26300374e+03, 6.36400364e+03,
 6.46500354e+03, 6.56600343e+03, 6.66700333e+03, 6.76800323e+03,
 6.86900313e+03, 6.97000303e+03, 7.07100293e+03, 7.17200283e+03,
 7.27300273e+03, 7.37400263e+03, 7.47500253e+03, 7.57600242e+03,
 7.67700232e+03, 7.77800222e+03, 7.87900212e+03, 7.98000202e+03,
 8.08100192e+03, 8.18200182e+03, 8.28300172e+03, 8.38400162e+03,
 8.48500152e+03, 8.58600141e+03, 8.68700131e+03, 8.78800121e+03,
 8.88900111e+03, 8.99000101e+03, 9.09100091e+03, 9.19200081e+03,
 9.29300071e+03, 9.39400061e+03, 9.49500051e+03, 9.59600040e+03,
 9.69700030e+03, 9.79800020e+03, 9.89900010e+03, 1.00000000e+04])

In [0]:
#The first two arguments of numpy.logspace are the exponents of the limits. Use
x = np.logspace(.01, 4, 100, endpoint=True)
x

Out[72]: array([1.02329299e+00, 1.12280162e+00, 1.23198682e+00, 1.35178957e+00,
 1.48324236e+00, 1.62747809e+00, 1.78573982e+00, 1.95939147e+00,
 2.14992962e+00, 2.35899637e+00, 2.58839351e+00, 2.84009805e+00,
 3.11627922e+00, 3.41931722e+00, 3.75182372e+00, 4.11666431e+00,
 4.51698330e+00, 4.95623072e+00, 5.43819211e+00, 5.96702113e+00,
 6.54727535e+00, 7.18395555e+00, 7.88254877e+00, 8.64907567e+00,
 9.49014236e+00, 1.04129973e+01, 1.14255940e+01, 1.25366591e+01,
 1.37557682e+01, 1.50934277e+01, 1.65611660e+01, 1.81716323e+01,
 1.99387060e+01, 2.18776162e+01, 2.40050730e+01, 2.63394111e+01,
 2.89007485e+01, 3.17111594e+01, 3.47948647e+01, 3.81784403e+01,
 4.18910467e+01, 4.59646802e+01, 5.04344481e+01, 5.53388721e+01,
 6.07202196e+01, 6.66248683e+01, 7.31037060e+01, 8.02125687e+01,
 8.80127224e+01, 9.65713905e+01, 1.05962334e+02, 1.16266486e+02,
 1.27572650e+02, 1.39978265e+02, 1.53590246e+02, 1.68525904e+02,
 1.84913959e+02, 2.02895646e+02, 2.22625936e+02, 2.44274868e+02,
 2.68029020e+02, 2.94093109e+02, 3.22691762e+02, 3.54071450e+02,
 3.88502609e+02, 4.26281975e+02, 4.67735141e+02, 5.13219359e+02,
 5.63126624e+02, 6.17887047e+02, 6.77972567e+02, 7.43901015e+02,
 8.16240579e+02, 8.95614698e+02, 9.82707436e+02, 1.07826938e+03,
 1.18312411e+03, 1.29817528e+03, 1.42441443e+03, 1.56292952e+03,
 1.71491431e+03, 1.88167864e+03, 2.06465972e+03, 2.26543453e+03,
 2.48573339e+03, 2.72745489e+03, 2.99268224e+03, 3.28370124e+03,
 3.60301995e+03, 3.95339034e+03, 4.33783199e+03, 4.75965810e+03,
 5.22250408e+03, 5.73035883e+03, 6.28759918e+03, 6.89902754e+03,
 7.56991336e+03, 8.30603849e+03, 9.11374703e+03, 1.00000000e+04])

In [0]:
xhfa.local.parametric_quantiles(params.load(), x)

Out[75]:

<xarray.Dataset>
Dimensions: (id: 5, scipy_dist: 7, return_period: 100)
Coordinates: (12/17)
 * id (id) object '020302' '020404' ... '020602' '020802'
 * scipy_dist (scipy_dist) <U11 'expon' 'gamma' ... 'weibull_min'
 drainage_area (id) float32 1.09e+03 647.0 59.8 626.0 1.2e+03
 end_date (id) datetime64[ns] 2006-10-13 ... 1996-08-13
 latitude (id) float32 48.77 48.81 48.98 48.98 49.2
 longitude (id) float32 -64.52 -64.92 -64.43 -64.7 -65.29
 ... ...
 start_date (id) datetime64[ns] 1989-08-12 ... 1970-01-01
 time_agg <U4 'mean'
 timestep <U1 'D'
 variable <U10 'streamflow'
 * return_period (return_period) float64 1.023 1.123 ... 1e+04
 p_quantile (return_period) float64 0.02276 0.1094 ... 0.9999
Data variables:
 streamflow_max_spring (scipy_dist, return_period, id) float64 0.278 ... ...
 streamflow_max_summer (scipy_dist, return_period, id) float64 0.04559 .....
 streamflow_max_fall (scipy_dist, return_period, id) float64 0.05592 .....
 streamflow_max_august (scipy_dist, return_period, id) float64 0.04559 .....
 streamflow_max_annual (scipy_dist, return_period, id) float64 0.278 ... ...
 streamflow_max_winter (scipy_dist, return_period, id) float64 0.1947 ......
Attributes:
 cat:frequency: yr
 cat:processing_level: indicators
 cat:id: xarray.Dataset Dimensions: id : 5 scipy_dist : 7 return_period : 100 Coordinates: (17) id (id) object '020302' '020404' ... '020802' cf_role : timeseries_id array(['020302', '020404', '020502', '020602', '020802'], dtype=object) scipy_dist (scipy_dist) <U11 'expon' 'gamma' ... 'weibull_min' array(['expon', 'gamma', 'genextreme', 'genpareto', 'gumbel_r', 'pearson3',
 'weibull_min'], dtype='<U11') drainage_area (id) float32 1.09e+03 647.0 59.8 626.0 1.2e+03 long_name : drainage_area units : km2 array([1090. , 647. , 59.8, 626. , 1200. ], dtype=float32) end_date (id) datetime64[ns] 2006-10-13 ... 1996-08-13 array(['2006-10-13T00:00:00.000000000', '2024-01-28T00:00:00.000000000',
 '1997-06-08T00:00:00.000000000', '2024-01-28T00:00:00.000000000',
 '1996-08-13T00:00:00.000000000'], dtype='datetime64[ns]') latitude (id) float32 48.77 48.81 48.98 48.98 49.2 long_name : latitude standard_name : latitude units : decimal_degrees array([48.769165, 48.806946, 48.982224, 48.97778 , 49.202778],
 dtype=float32) longitude (id) float32 -64.52 -64.92 -64.43 -64.7 -65.29 long_name : longitude standard_name : longitude units : decimal_degrees array([-64.51583 , -64.916664, -64.42694 , -64.69972 , -65.29472 ],
 dtype=float32) name (id) object 'Saint' 'York' ... 'Madeleine' array(['Saint', 'York', 'Au Renard', 'Dartmouth', 'Madeleine'],
 dtype=object) province (id) object 'QC' 'QC' 'QC' 'QC' 'QC' array(['QC', 'QC', 'QC', 'QC', 'QC'], dtype=object) regulated (id) object 'Natural' 'Natural' ... 'Natural' array(['Natural', 'Natural', 'Natural', 'Natural', 'Natural'],
 dtype=object) source () <U102 'Ministère de l’Environnement, d... array('Ministère de l’Environnement, de la Lutte contre les changements climatiques, de la Faune et des Parcs',
 dtype='<U102') spatial_agg () <U9 'watershed' array('watershed', dtype='<U9') start_date (id) datetime64[ns] 1989-08-12 ... 1970-01-01 array(['1989-08-12T00:00:00.000000000', '1980-10-01T00:00:00.000000000',
 '1977-07-10T00:00:00.000000000', '1970-10-01T00:00:00.000000000',
 '1970-01-01T00:00:00.000000000'], dtype='datetime64[ns]') time_agg () <U4 'mean' array('mean', dtype='<U4') timestep () <U1 'D' array('D', dtype='<U1') variable () <U10 'streamflow' array('streamflow', dtype='<U10') return_period (return_period) float64 1.023 1.123 ... 9.114e+03 1e+04 array([1.023293e+00, 1.122802e+00, 1.231987e+00, 1.351790e+00, 1.483242e+00,
 1.627478e+00, 1.785740e+00, 1.959391e+00, 2.149930e+00, 2.358996e+00,
 2.588394e+00, 2.840098e+00, 3.116279e+00, 3.419317e+00, 3.751824e+00,
 4.116664e+00, 4.516983e+00, 4.956231e+00, 5.438192e+00, 5.967021e+00,
 6.547275e+00, 7.183956e+00, 7.882549e+00, 8.649076e+00, 9.490142e+00,
 1.041300e+01, 1.142559e+01, 1.253666e+01, 1.375577e+01, 1.509343e+01,
 1.6

In [0]:
xhfa.local.parametric_quantiles(params, x).load()

Out[74]:

<xarray.Dataset>
Dimensions: (id: 5, scipy_dist: 7, return_period: 100)
Coordinates: (12/17)
 * id (id) object '020302' '020404' ... '020602' '020802'
 * scipy_dist (scipy_dist) <U11 'expon' 'gamma' ... 'weibull_min'
 drainage_area (id) float32 1.09e+03 647.0 59.8 626.0 1.2e+03
 end_date (id) datetime64[ns] 2006-10-13 ... 1996-08-13
 latitude (id) float32 48.77 48.81 48.98 48.98 49.2
 longitude (id) float32 -64.52 -64.92 -64.43 -64.7 -65.29
 ... ...
 start_date (id) datetime64[ns] 1989-08-12 ... 1970-01-01
 time_agg <U4 'mean'
 timestep <U1 'D'
 variable <U10 'streamflow'
 * return_period (return_period) float64 1.023 1.123 ... 1e+04
 p_quantile (return_period) float64 0.02276 0.1094 ... 0.9999
Data variables:
 streamflow_max_spring (scipy_dist, return_period, id) float64 0.278 ... ...
 streamflow_max_summer (scipy_dist, return_period, id) float64 0.04559 .....
 streamflow_max_fall (scipy_dist, return_period, id) float64 0.05592 .....
 streamflow_max_august (scipy_dist, return_period, id) float64 0.04559 .....
 streamflow_max_annual (scipy_dist, return_period, id) float64 0.278 ... ...
 streamflow_max_winter (scipy_dist, return_period, id) float64 0.1947 ......
Attributes:
 cat:frequency: yr
 cat:processing_level: indicators
 cat:id: xarray.Dataset Dimensions: id : 5 scipy_dist : 7 return_period : 100 Coordinates: (17) id (id) object '020302' '020404' ... '020802' cf_role : timeseries_id array(['020302', '020404', '020502', '020602', '020802'], dtype=object) scipy_dist (scipy_dist) <U11 'expon' 'gamma' ... 'weibull_min' array(['expon', 'gamma', 'genextreme', 'genpareto', 'gumbel_r', 'pearson3',
 'weibull_min'], dtype='<U11') drainage_area (id) float32 1.09e+03 647.0 59.8 626.0 1.2e+03 long_name : drainage_area units : km2 array([1090. , 647. , 59.8, 626. , 1200. ], dtype=float32) end_date (id) datetime64[ns] 2006-10-13 ... 1996-08-13 array(['2006-10-13T00:00:00.000000000', '2024-01-28T00:00:00.000000000',
 '1997-06-08T00:00:00.000000000', '2024-01-28T00:00:00.000000000',
 '1996-08-13T00:00:00.000000000'], dtype='datetime64[ns]') latitude (id) float32 48.77 48.81 48.98 48.98 49.2 long_name : latitude standard_name : latitude units : decimal_degrees array([48.769165, 48.806946, 48.982224, 48.97778 , 49.202778],
 dtype=float32) longitude (id) float32 -64.52 -64.92 -64.43 -64.7 -65.29 long_name : longitude standard_name : longitude units : decimal_degrees array([-64.51583 , -64.916664, -64.42694 , -64.69972 , -65.29472 ],
 dtype=float32) name (id) object 'Saint' 'York' ... 'Madeleine' array(['Saint', 'York', 'Au Renard', 'Dartmouth', 'Madeleine'],
 dtype=object) province (id) object 'QC' 'QC' 'QC' 'QC' 'QC' array(['QC', 'QC', 'QC', 'QC', 'QC'], dtype=object) regulated (id) object 'Natural' 'Natural' ... 'Natural' array(['Natural', 'Natural', 'Natural', 'Natural', 'Natural'],
 dtype=object) source () <U102 'Ministère de l’Environnement, d... array('Ministère de l’Environnement, de la Lutte contre les changements climatiques, de la Faune et des Parcs',
 dtype='<U102') spatial_agg () <U9 'watershed' array('watershed', dtype='<U9') start_date (id) datetime64[ns] 1989-08-12 ... 1970-01-01 array(['1989-08-12T00:00:00.000000000', '1980-10-01T00:00:00.000000000',
 '1977-07-10T00:00:00.000000000', '1970-10-01T00:00:00.000000000',
 '1970-01-01T00:00:00.000000000'], dtype='datetime64[ns]') time_agg () <U4 'mean' array('mean', dtype='<U4') timestep () <U1 'D' array('D', dtype='<U1') variable () <U10 'streamflow' array('streamflow', dtype='<U10') return_period (return_period) float64 1.023 1.123 ... 9.114e+03 1e+04 array([1.023293e+00, 1.122802e+00, 1.231987e+00, 1.351790e+00, 1.483242e+00,
 1.627478e+00, 1.785740e+00, 1.959391e+00, 2.149930e+00, 2.358996e+00,
 2.588394e+00, 2.840098e+00, 3.116279e+00, 3.419317e+00, 3.751824e+00,
 4.116664e+00, 4.516983e+00, 4.956231e+00, 5.438192e+00, 5.967021e+00,
 6.547275e+00, 7.183956e+00, 7.882549e+00, 8.649076e+00, 9.490142e+00,
 1.041300e+01, 1.142559e+01, 1.253666e+01, 1.375577e+01, 1.509343e+01,
 1.6

In [0]:
timeit.timeit(lambda: xr.apply_ufunc(vec_plotting_positions, ds_4fa, input_core_dims=[['time']], output_core_dims=[['time']]), number=5000)

Out[47]: 10.606597564000367

In [0]:
timeit.timeit(lambda: xr.apply_ufunc(plotting_positions, ds_4fa, input_core_dims=[['time']], output_core_dims=[['time']], vectorize=True), number=5000)

Out[48]: 17.22995357899981